<a href="https://colab.research.google.com/github/alfazick/AppliedLLMCourse/blob/main/ExtraTopic-function_call_finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Lesson Learned: Use Transformers

In [ ]:
import os
os.environ["UNSLOTH_DISABLE_GRADIENT_OFFLOAD"] = "1"

In [ ]:
import torch
from unsloth import FastLanguageModel

print(torch.cuda.get_device_name())

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/opt/conda/lib/python3.11/site-packages/transformers/utils/hub.py:110: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


INFO 11-26 06:34:23 [__init__.py:216] Automatically detected platform cuda.
🦥 Unsloth Zoo will now patch everything to make training faster!
NVIDIA GeForce RTX 5090


In [ ]:
import torch; print(torch.cuda.get_device_capability())

(12, 0)


In [ ]:
# ok so goal of my project to fine-tune the model
# for function calling
# 1) first direct
# 2) if time allows introduce new special token, something like <tool>
# 3) create evaluation, like start with simple hit or miss

In [ ]:
import os
import wandb
from dotenv import load_dotenv
load_dotenv()

def setup_wandb(project_name: str, run_name: str):
    api_key = os.getenv("WANDB_API_KEY")
    if not api_key:
        raise EnvironmentError("WANDB_API_KEY is not set")

    wandb.login(key=api_key)

    os.environ["WANDB_LOG_MODEL"] = "checkpoint"
    os.environ["WANDB_WATCH"] = "all"
    os.environ["WANDB_SILENT"] = "true"

    wandb.init(project=project_name, name=run_name)

setup_wandb(project_name="function-calling-finetune", run_name="llama3.1-8b-xlam-v1")


wandb: ERROR Failed to detect the name of this notebook. You can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/unsloth/.netrc
wandb: Currently logged in as: alfazick (askarnlp) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


wandb: Detected [huggingface_hub.inference, openai] in use.
wandb: Use W&B Weave for improved LLM call tracing. Install Weave with `pip install weave` then add `import weave` to the top of your script.
wandb: For more information, check out the docs at: https://weave-docs.wandb.ai/


In [ ]:
#https://huggingface.co/settings/tokens
load_dotenv()
from huggingface_hub import login
hf_token = os.getenv("HUGGINGFACE_TOKEN")
if hf_token is None:
    raise EnvironmentError("HUGGINGFACE_TOKEN is not set in the environment variables.")
login(hf_token)

In [ ]:
import torch
from unsloth import FastLanguageModel

max_seq_length = 2048     # Unsloth auto supports RoPE Scaling internally!
dtype = None              # None for auto detection
load_in_4bit = False      # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B-Instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    attn_implementation="flash_attention_2",
)

==((====))==  Unsloth 2025.11.2: Fast Llama patching. Transformers: 4.57.0. vLLM: 0.10.2.
   \\   /|    NVIDIA GeForce RTX 5090. Num GPUs = 1. Max memory: 31.367 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu128. CUDA: 12.0. CUDA Toolkit: 12.8. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33+aa7bc36.d20251112. FA2 = True]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,   # LoRA rank - suggested values: 8, 16, 32, 64, 128
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj"],
    lora_alpha=16,
    lora_dropout=0,   # Supports any, but = 0 is optimized
    bias="none",      # Supports any, but = "none" is optimized
    use_gradient_checkpointing=True,
    random_state=3407,
    use_rslora=False,   # Disable rank-sensitive LoRA for simpler tasks
    loftq_config=None   # No LoftQ, for standard fine-tuning
)

Unsloth 2025.11.2 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
from datasets import load_dataset

# Loading the dataset
dataset = load_dataset("Salesforce/xlam-function-calling-60k", split="train", token=hf_token)

# Selecting a subset of 15K samples for fine-tuning
dataset = dataset.select(range(1000))
print(f"Using a sample size of {len(dataset)} for fine-tuning.")


Using a sample size of 1000 for fine-tuning.


In [ ]:
from unsloth.chat_templates import get_chat_template
# Initialize the tokenizer with the chat template and mapping
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3",
    mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt"}, # ShareGPT style
    map_eos_token = True,        # Maps <|im_end|> to <|eot_id|> instead
)

def formatting_prompts_func(examples):
    convos = []

    # Iterate through each item in the batch (examples are structured as lists of values)
    for query, tools, answers in zip(examples['query'], examples['tools'], examples['answers']):
        tool_user = {
            "content": f"You are a helpful assistant with access to the following tools or function calls. Your task is to produce a sequence of tools or function calls necessary to generate response to the user utterance. Use the following tools or function calls as required:\n{tools}",
            "role": "system"
        }
        ques_user = {
            "content": f"{query}",
            "role": "user"
        }
        assistant = {
            "content": f"{answers}",
            "role": "assistant"
        }
        convos.append([tool_user, ques_user, assistant])

    texts = [tokenizer.apply_chat_template(convo, tokenize=False, add_generation_prompt=False) for convo in convos]
    return {"text": texts}

# Apply the formatting on dataset
dataset = dataset.map(formatting_prompts_func, batched = True,)


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
print(dataset[0]["text"])

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a helpful assistant with access to the following tools or function calls. Your task is to produce a sequence of tools or function calls necessary to generate response to the user utterance. Use the following tools or function calls as required:
[{"name": "live_giveaways_by_type", "description": "Retrieve live giveaways from the GamerPower API based on the specified type.", "parameters": {"type": {"description": "The type of giveaways to retrieve (e.g., game, loot, beta).", "type": "str", "default": "game"}}}]<|eot_id|><|start_header_id|>user<|end_header_id|>

Where can I find live giveaways for beta access and games?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

[{"name": "live_giveaways_by_type", "arguments": {"type": "beta"}}, {"name": "live_giveaways_by_type", "arguments": {"type": "game"}}]<|eot_id|>


In [ ]:
# Split the dataset
dataset = dataset.train_test_split(test_size=0.1, seed=3407)

print(f"Train: {len(dataset['train'])} samples")
print(f"Validation: {len(dataset['test'])} samples")

Train: 900 samples
Validation: 100 samples


In [ ]:
from transformers import TrainingArguments

args = TrainingArguments(
        per_device_train_batch_size = 8,  # Controls the batch size per device
        gradient_accumulation_steps = 2,  # Accumulates gradients to simulate a larger batch
        warmup_steps = 5,
        learning_rate = 2e-4,             # Sets the learning rate for optimization
        num_train_epochs = 1,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        optim = "adamw_8bit",
        weight_decay = 0.01,              # Regularization term for preventing overfitting
        lr_scheduler_type = "linear",     # Chooses a linear learning rate decay
        seed = 3407,
        output_dir = "outputs",
        report_to = "wandb",              # Enables Weights & Biases (W&B) logging
        logging_steps = 10,                # Sets frequency of logging to W&B
        logging_strategy = "steps",       # Logs metrics at each specified step
        save_strategy = "epoch",
        eval_strategy="epoch",
        load_best_model_at_end = True,    # Loads the best model at the end
        save_only_model = False           # Saves entire model, not only weights
    )


In [ ]:
from trl import SFTTrainer
trainer = SFTTrainer(
    model = model,
    processing_class = tokenizer,
    train_dataset = dataset["train"],
    eval_dataset=dataset["test"],
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,        # Can make training 5x faster for short sequences.
    args = args
)

Unsloth: Tokenizing ["text"] (num_proc=124):   0%|          | 0/900 [00:00<?, ? examples/s]

num_proc must be <= 100. Reducing num_proc to 100 for dataset of size 100.
[datasets.arrow_dataset|WARNING]num_proc must be <= 100. Reducing num_proc to 100 for dataset of size 100.


Unsloth: Tokenizing ["text"] (num_proc=100):   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
# Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")


GPU = NVIDIA GeForce RTX 5090. Max memory = 31.367 GB.
15.158 GB of memory reserved.


In [ ]:
from unsloth import unsloth_train

trainer_stats = unsloth_train(trainer)
print(trainer_stats)
wandb.finish()

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 900 | Num Epochs = 1 | Total steps = 57
O^O/ \_/ \    Batch size per device = 8 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (8 x 2 x 1) = 16
 "-____-"     Trainable parameters = 41,943,040 of 8,072,204,288 (0.52% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Epoch,Training Loss,Validation Loss
1,0.660500,0.626902


Unsloth: Not an error, but LlamaForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient
wandb: Adding directory to artifact (outputs/checkpoint-57)... Done. 0.5s


TrainOutput(global_step=57, training_loss=0.826810267933628, metrics={'train_runtime': 531.0377, 'train_samples_per_second': 1.695, 'train_steps_per_second': 0.107, 'total_flos': 3.256334555362099e+16, 'train_loss': 0.826810267933628, 'epoch': 1.0})


eval/loss,▁
eval/runtime,▁
eval/samples_per_second,▁
eval/steps_per_second,▁
train/epoch,▁▃▄▆▇██
train/global_step,▁▂▄▅▇██
train/grad_norm,█▁▁▁▂
train/learning_rate,█▆▄▃▁
train/loss,█▂▁▁▁
eval/loss,0.6269
eval/runtime,9.7311


In [ ]:
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")




531.0377 seconds used for training.
8.85 minutes used for training.
Peak reserved memory = 24.336 GB.
Peak reserved memory for training = 9.178 GB.
Peak reserved memory % of max memory = 77.585 %.
Peak reserved memory for training % of max memory = 29.26 %.


In [ ]:
FastLanguageModel.for_inference(model)

import json

query = "What's the weather in Tokyo?"

tools = [{"name": "get_weather", "description": "Get current weather for a location", "parameters": {"location": {"description": "The city name", "type": "str", "default": "San Francisco"}}}]

chat = [
    {"role": "system", "content": f"You are a helpful assistant with access to the following tools or function calls. Your task is to produce a sequence of tools or function calls necessary to generate response to the user utterance. Use the following tools or function calls as required:\n{json.dumps(tools)}"},
    {"role": "user", "content": query}
]

inputs = tokenizer.apply_chat_template(chat, tokenize=True, add_generation_prompt=True, return_tensors="pt").to("cuda")
outputs = model.generate(inputs, max_new_tokens=128, use_cache=True)
print(tokenizer.decode(outputs[0], skip_special_tokens=False))

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a helpful assistant with access to the following tools or function calls. Your task is to produce a sequence of tools or function calls necessary to generate response to the user utterance. Use the following tools or function calls as required:
[{"name": "get_weather", "description": "Get current weather for a location", "parameters": {"location": {"description": "The city name", "type": "str", "default": "San Francisco"}}}]<|eot_id|><|start_header_id|>user<|end_header_id|>

What's the weather in Tokyo?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

[{"name": "get_weather", "arguments": {"location": "Tokyo"}}]<|eot_id|>


In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'query', 'answers', 'tools', 'text'],
        num_rows: 900
    })
    test: Dataset({
        features: ['id', 'query', 'answers', 'tools', 'text'],
        num_rows: 100
    })
})

In [ ]:
import json
from tqdm import tqdm

FastLanguageModel.for_inference(model)

def evaluate(dataset, max_samples=None):
    correct = 0
    total = 0
    results = []

    samples = dataset if max_samples is None else dataset.select(range(min(max_samples, len(dataset))))

    for example in tqdm(samples):
        # Build prompt from tools and query
        chat = [
            {"role": "system", "content": f"You are a helpful assistant with access to the following tools or function calls. Your task is to produce a sequence of tools or function calls necessary to generate response to the user utterance. Use the following tools or function calls as required:\n{example['tools']}"},
            {"role": "user", "content": example['query']}
        ]

        inputs = tokenizer.apply_chat_template(chat, tokenize=True, add_generation_prompt=True, return_tensors="pt").to("cuda")
        outputs = model.generate(inputs, max_new_tokens=256, use_cache=True, pad_token_id=tokenizer.eos_token_id)

        # Extract just the generated part
        response = tokenizer.decode(outputs[0][inputs.shape[1]:], skip_special_tokens=True).strip()

        # Compare
        expected = example['answers']

        # Try parsing both as JSON for comparison
        try:
            pred_json = json.loads(response)
            expected_json = json.loads(expected)
            match = pred_json == expected_json
        except:
            match = response == expected

        if match:
            correct += 1

        results.append({
            "query": example['query'],
            "expected": expected,
            "predicted": response,
            "match": match
        })
        total += 1

    accuracy = correct / total * 100
    print(f"\nAccuracy: {correct}/{total} = {accuracy:.2f}%")

    return accuracy, results

# Run eval
accuracy, results = evaluate(dataset['test'])

# Show some failures
print("\n--- Sample Failures ---")
for r in results[:20]:
    if not r['match']:
        print(f"Q: {r['query'][:80]}...")
        print(f"Expected: {r['expected'][:80]}...")
        print(f"Got:      {r['predicted'][:80]}...")
        print()

100%|██████████| 100/100 [03:05<00:00,  1.85s/it]


Accuracy: 62/100 = 62.00%

--- Sample Failures ---
Q: Can you provide the hourly forecast for New York City tomorrow?...
Expected: [{"name": "get_hourly_forecast", "arguments": {"city": "New York City"}}]...
Got:      [{"name": "get_hourly_forecast", "arguments": {"city": "New York"}}]...

Q: Can you provide the Stochastic RSI for AAPL stock over the last month using dail...
Expected: [{"name": "stochrsi", "arguments": {"symbol": "AAPL", "interval": "1day", "outpu...
Got:      [{"name": "stochrsi", "arguments": {"symbol": "AAPL", "interval": "1day", "fast_...

Q: Fetch the treasure of the day from the Uncovered Treasure API....
Expected: [{"name": "today", "arguments": {}}]...
Got:      [{"name": "today", "arguments": {"callback": ""}}]...



In [ ]:
import json
from tqdm import tqdm

def evaluate_flexible(dataset, max_samples=None):
    metrics = {"name_match": 0, "args_match": 0, "exact_match": 0, "parse_fail": 0}
    total = 0
    results = []

    samples = dataset if max_samples is None else dataset.select(range(min(max_samples, len(dataset))))

    for example in tqdm(samples):
        chat = [
            {"role": "system", "content": f"You are a helpful assistant with access to the following tools or function calls. Your task is to produce a sequence of tools or function calls necessary to generate response to the user utterance. Use the following tools or function calls as required:\n{example['tools']}"},
            {"role": "user", "content": example['query']}
        ]

        inputs = tokenizer.apply_chat_template(chat, tokenize=True, add_generation_prompt=True, return_tensors="pt").to("cuda")
        outputs = model.generate(inputs, max_new_tokens=256, use_cache=True, pad_token_id=tokenizer.eos_token_id)
        response = tokenizer.decode(outputs[0][inputs.shape[1]:], skip_special_tokens=True).strip()

        expected = example['answers']

        try:
            pred = json.loads(response)
            exp = json.loads(expected)

            # Normalize to list
            if isinstance(pred, dict): pred = [pred]
            if isinstance(exp, dict): exp = [exp]

            # Function name match (most important)
            pred_names = {p.get('name') for p in pred}
            exp_names = {e.get('name') for e in exp}
            name_match = pred_names == exp_names

            # Arguments match (for matching functions)
            args_match = pred == exp  # strict

            if name_match: metrics["name_match"] += 1
            if args_match: metrics["args_match"] += 1
            if pred == exp: metrics["exact_match"] += 1

        except json.JSONDecodeError:
            metrics["parse_fail"] += 1
            name_match = False
            args_match = False

        results.append({"query": example['query'], "expected": expected, "predicted": response, "name_match": name_match})
        total += 1

    print(f"\n=== Results (n={total}) ===")
    print(f"JSON Parse Success: {total - metrics['parse_fail']}/{total} ({(total-metrics['parse_fail'])/total*100:.1f}%)")
    print(f"Function Name Match: {metrics['name_match']}/{total} ({metrics['name_match']/total*100:.1f}%)")
    print(f"Exact Match: {metrics['exact_match']}/{total} ({metrics['exact_match']/total*100:.1f}%)")

    return metrics, results

metrics, results = evaluate_flexible(dataset['test'])

100%|██████████| 100/100 [03:16<00:00,  1.97s/it]


=== Results (n=100) ===
JSON Parse Success: 97/100 (97.0%)
Function Name Match: 95/100 (95.0%)
Exact Match: 62/100 (62.0%)


In [ ]:
def evaluate_detailed(dataset, max_samples=None):
    metrics = {"precision": [], "recall": [], "f1": [], "exact": 0, "parse_fail": 0}
    total = 0

    samples = dataset if max_samples is None else dataset.select(range(min(max_samples, len(dataset))))

    for example in tqdm(samples):
        chat = [
            {"role": "system", "content": f"You are a helpful assistant with access to the following tools or function calls. Your task is to produce a sequence of tools or function calls necessary to generate response to the user utterance. Use the following tools or function calls as required:\n{example['tools']}"},
            {"role": "user", "content": example['query']}
        ]

        inputs = tokenizer.apply_chat_template(chat, tokenize=True, add_generation_prompt=True, return_tensors="pt").to("cuda")
        outputs = model.generate(inputs, max_new_tokens=256, use_cache=True, pad_token_id=tokenizer.eos_token_id)
        response = tokenizer.decode(outputs[0][inputs.shape[1]:], skip_special_tokens=True).strip()

        try:
            pred = json.loads(response)
            exp = json.loads(example['answers'])

            if isinstance(pred, dict): pred = [pred]
            if isinstance(exp, dict): exp = [exp]

            pred_names = {p.get('name') for p in pred}
            exp_names = {e.get('name') for e in exp}

            # Precision: what % of predicted were correct?
            if len(pred_names) > 0:
                precision = len(pred_names & exp_names) / len(pred_names)
            else:
                precision = 0

            # Recall: what % of expected were called?
            if len(exp_names) > 0:
                recall = len(pred_names & exp_names) / len(exp_names)
            else:
                recall = 1

            # F1
            if precision + recall > 0:
                f1 = 2 * precision * recall / (precision + recall)
            else:
                f1 = 0

            metrics["precision"].append(precision)
            metrics["recall"].append(recall)
            metrics["f1"].append(f1)
            if pred == exp: metrics["exact"] += 1

        except:
            metrics["parse_fail"] += 1

        total += 1

    valid = total - metrics["parse_fail"]
    print(f"\n=== Results (n={total}) ===")
    print(f"JSON Parse: {valid}/{total} ({valid/total*100:.1f}%)")
    print(f"Precision:  {sum(metrics['precision'])/len(metrics['precision'])*100:.1f}%  (no extra calls)")
    print(f"Recall:     {sum(metrics['recall'])/len(metrics['recall'])*100:.1f}%  (no missing calls)")
    print(f"F1:         {sum(metrics['f1'])/len(metrics['f1'])*100:.1f}%")
    print(f"Exact:      {metrics['exact']}/{total} ({metrics['exact']/total*100:.1f}%)")

    return metrics

metrics = evaluate_detailed(dataset['test'])

100%|██████████| 100/100 [03:15<00:00,  1.95s/it]


=== Results (n=100) ===
JSON Parse: 96/100 (96.0%)
Precision:  99.0%  (no extra calls)
Recall:     99.0%  (no missing calls)
F1:         99.0%
Exact:      63/100 (63.0%)


In [ ]:
model.push_to_hub_merged(
    "alfazick/llama-3.1-8b-function-calling",
    tokenizer,
    save_method="merged_16bit",
    token=hf_token
)

Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

No existing and accessible Hugging Face cache directory found.


Unsloth: Preparing safetensor model files: 100%|██████████| 4/4 [00:41<00:00, 10.38s/it]


Note: tokenizer.model not found (this is OK for non-SentencePiece models)


...

Unsloth: Merging weights into 16bit:  75%|███████▌  | 3/4 [03:11<01:03, 63.67s/it]

Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

Unsloth: Merging weights into 16bit: 100%|██████████| 4/4 [03:25<00:00, 51.38s/it]


Unsloth: Merge process complete. Saved to `/workspace/work/alfazick/llama-3.1-8b-function-calling`


In [ ]:
# conclusion: don't use unsloth, easier to use native TRL
"""I used Unsloth initially, discovered their gradient offloading bottlenecks high-VRAM GPUs.
Diagnosed with nvidia-smi — 0% GPU, 90% CPU.
The 'smart' offloading triggers regardless of available memory.
Would use pure PEFT + Flash Attention 2 next time."""